In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [6]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = '20171013_wp248_rev_01_en_D7D5A266-FAE9-3CA1-65B7371E82EE1891_47711.pdf'
paper_name = 'Guidelines on Data Protection Impact Assessment (DPIA) and determining whether processing is “likely to result in a high risk” for the purposes of Regulation 2016/679'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

In [12]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=90, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

file = "../tmp/dpia.txt"
with open(file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/dpia.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""
section_pattern = re.compile(r'\b(I|II|III|IV|)\.\s*(.*)')
subsection_pattern = re.compile(r'([A-Z])\.\s*(.*)') 
point_pattern = re.compile(r'([a-z])\)\s*(.*)$') 
# subpoint_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX (\d+) - (.+)$') # ignore capitalization

table_pattern = re.compile(r'\|')
# table_entry_number_pattern = re.compile(r'(\d+(\.\d+)?(\.\d+)?)\s+(.+)')
# table_section = re.compile(r'<td colspan=6>(\d+) - (.+)$')
# table_subsection = re.compile(r'<td colspan=6>\s*(.+)\s*')


data = []
table_data = []
for entry in doc_as_array:
    section_match = section_pattern.match(entry)
    subsection_match = subsection_pattern.match(entry)
    point_match = point_pattern.match(entry)
    #subpoint_match = subpoint_pattern.match(entry)
    annex_match = annex_pattern.match(entry)
    table_match = table_pattern.match(entry)
    if section_match:
        match = section_match
        section = match.group(1)
        subsection = ""
        point = ""
        #subpoint = ""
        heading = True
        text = match.group(2)
        section_reference = section
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
    elif subsection_match:
        match = subsection_match
        section = section
        subsection = match.group(1)
        point = ""
        #subpoint = ""
        heading = True
        text = match.group(2)
        section_reference = section + "." + subsection
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
    elif point_match:
        match = point_match
        section = section
        subsection = subsection
        point = match.group(1)
        #subpoint = ""
        heading = True
        text = match.group(2)
        section_reference = section + "." + subsection + "." + point
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
    # elif subpoint_match:
    #     match = subpoint_match
    #     section = section
    #     subsection = subsection
    #     point = point
    #     subpoint = match.group(1)
    #     heading = False
    #     text = match.group(2)
    #     section_reference = section + "." + subsection + "." + point + ")" + subpoint
    #     data.append([section, subsection, point, subpoint, heading, text, section_reference])
    
    
    elif annex_match:
        match = annex_match
        section = "Annex " + match.group(1)
        subsection = ""
        point = ""
        #subpoint = ""
        heading = True
        text = match.group(2)
        section_reference = section
        #data.append([section, subsection, point, subpoint, heading, text, section_reference])
        data.append([section, subsection, point, heading, text, section_reference])
    # elif table_match:
    #     pass
        # components = [component.strip() for component in entry.strip().strip('|').split('|')]
        # if len(components) == 5:
        #     table_entry_number_match = table_entry_number_pattern.match(components[0])
        #     if table_entry_number_match:
        #         components = [table_entry_number_match.group(1), table_entry_number_match.group(4)] + components[1:]
        #         table_data.append(components)
        #     else:
        #         print(f"Row removed from table_data: {components}")
        #         #components = [""] + components
        # elif len(components) == 1:
        #     table_section_match = table_section.match(components[0])
        #     table_subsection_match = table_subsection.match(components[0])
        #     if table_section_match:
        #         components = [table_section_match.group(1), table_section_match.group(2)]
        #         table_data.append(components)
        #     elif table_subsection_match:
        #         print(f"Row removed from table_data: {components}")
        #         # components = ["", table_subsection_match.group(1)]
        #     else:
        #         raise AttributeError(f"A row with one element did not match the required patterns: {entry}")
        # else:
        #     raise AttributeError("A table entry did not have the correct number of elements")
    else:
        section = section
        subsection = subsection
        point = point
        heading = False
        text = entry
        section_reference = section
        if subsection:
            if point:
                section_reference = section + "." + subsection + "." + point
            else:
                section_reference = section + "." + subsection
        
        #data.append(["", "", "", "",  heading, text, section_reference])
        data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[60:80]

,section,subsection,point,heading,text,section_reference
60,,,,False,|An e-commerce website displaying adverts for ...,III.B.a
61,,,,False,"Conversely, a processing operation may corresp...",III.B.a
62,,,,False,"In addition, as part of the accountability pri...",III.B.a
63,,,,False,Note: supervisory authorities are required to ...,III.B.a
64,III,B,b,True,When isn't a DPIA required? When the processin...,III.B.b
65,,,,False,WP29 considers that a DPIA is not required in ...,III.B.b
66,,,,False,"- where the processing is not ""likely to resul...",III.B.b
67,,,,False,"- when the nature, scope, context and purposes...",III.B.b
68,,,,False,- when the processing operations have been che...,III.B.b
69,,,,False,"- where a processing operation, pursuant to po...",III.B.b


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 2 augmented with footnotes
Row 6 augmented with footnotes
Row 7 augmented with footnotes
Row 8 augmented with footnotes
Row 9 augmented with footnotes
Row 18 augmented with footnotes
Row 32 augmented with footnotes
Row 33 augmented with footnotes
Row 34 augmented with footnotes
Row 37 augmented with footnotes
Row 40 augmented with footnotes
Row 42 augmented with footnotes
Row 43 augmented with footnotes
Row 63 augmented with footnotes
Row 67 augmented with footnotes
Row 68 augmented with footnotes
Row 69 augmented with footnotes
Row 75 augmented with footnotes
Row 79 augmented with footnotes
Row 92 augmented with footnotes
Row 103 augmented with footnotes
Row 105 augmented with footnotes
Row 113 augmented with footnotes
Row 118 augmented with footnotes
Row 122 augmented with footnotes
Row 125 augmented with footnotes
Row 144 augmented with footnotes
Row 149 augmented with footnotes
Row 150 augmented with footnotes


In [6]:
file = "../../inputs/documents/dpia.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [1]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.dpia
importlib.reload(gdpr_rag.documents.dpia)
from gdpr_rag.documents.dpia import DPIA

path_to_manual_as_csv_file = "../../inputs/documents/dpia.parquet"

doc = DPIA(path_to_manual_as_csv_file)


In [8]:
from IPython.display import Markdown, display

section = "III.B.a"
# section = "III.B"

# section = "Annex 1"

# section = "IV.E.1"
# section = "III.A.1"
# section = "III.D.4"

# section = "IV"
# section = "IV.E"
# section = "IV.E.1"


doc.get_heading(section)
doc.get_text(section)

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


III DPIA: the Regulation explained
III.B Which processing operations are subject to a DPIA? Apart from exceptions, where they are "likely to result in a high risk".
III.B.a When is a DPIA mandatory? When processing is "likely to result in a high risk".


# III DPIA: the Regulation explained

## III.B Which processing operations are subject to a DPIA? Apart from exceptions, where they are "likely to result in a high risk".

### III.B.a When is a DPIA mandatory? When processing is "likely to result in a high risk".

The GDPR does not require a DPIA to be carried out for every processing operation which may result in risks for the rights and freedoms of natural persons. The carrying out of a DPIA is only mandatory where processing is "likely to result in a high risk to the rights and freedoms of natural persons" (Article 35(1), illustrated by Article 35(3) and complemented by Article 35(4)). It is particularly relevant when a new data processing technology is being introduced[^11]. In cases where it is not clear whether a DPIA is required, the WP29 recommends that a DPIA is carried out nonetheless as a DPIA is a useful tool to help controllers comply with data protection law. Even though a DPIA could be required in other circumstances, Article 35(3) provides some examples when a processing operation is "likely to result in high risks":

(a) a systematic and extensive evaluation of personal aspects relating to natural persons which is based on automated processing, including profiling, and on which decisions are based that produce legal effects concerning the natural person or similarly significantly affect the natural person[^12];

(b) processing on a large scale of special categories of data referred to in Article 9(1), or of personal data relating to criminal convictions and offences referred to in Article 10[^13]; or

(c) a systematic monitoring of a publicly accessible area on a large scale".

As the words "in particular" in the introductory sentence of Article 35(3) GDPR indicate, this is meant as a non-exhaustive list. There may be "high risk" processing operations that are not captured by this list, but yet pose similarly high risks. Those processing operations should also be subject to DPIAs. For this reason, the criteria developed below sometimes go beyond a simple explanation of what should be understood by the three examples given in Article 35(3) GDPR.

In order to provide a more concrete set of processing operations that require a DPIA due to their inherent high risk, taking into account the particular elements of Articles 35(1) and 35(3)(a) to (c), the list to be adopted at the national level under article 35(4) and recitals 71, 75 and 91, and other GDPR references to "likely to result in a high risk" processing operations[^14], the following nine criteria should be considered.

1. Evaluation or scoring, including profiling and predicting, especially from "aspects concerning the data subject's performance at work, economic situation, health, personal preferences or interests, reliability or behavior, location or movements" (recitals 71 and 91). Examples of this could include a financial institution that screens its customers against a credit reference database or against an anti-money laundering and counter-terrorist financing (AML/CTF) or fraud database, or a biotechnology company offering genetic tests directly to consumers in order to assess and predict the disease/health risks, or a company building behavioural or marketing profiles based on usage or navigation on its website.

2. Automated-decision making with legal or similar significant effect: processing that aims at taking decisions on data subjects producing "legal effects concerning the natural person" or which "similarly significantly affects the natural person" (Article 35(3)(a)). For example, the processing may lead to the exclusion or discrimination against individuals. Processing with little or no effect on individuals does not match this specific criterion. Further explanations on these notions will be provided in the upcoming WP29 Guidelines on Profiling.

3. Systematic monitoring: processing used to observe, monitor or control data subjects, including data collected through networks or "a systematic monitoring of a publicly accessible area" (Article 35(3)(c)).[^15] This type of monitoring is a criterion because the personal data may be collected in circumstances where data subjects may not be aware of who is collecting their data and how they will be used. Additionally, it may be impossible for individuals to avoid being subject to such processing in public (or publicly accessible) space(s).

4. Sensitive data or data of a highly personal nature: this includes special categories of personal data as defined in Article 9 (for example information about individuals' political opinions), as well as personal data relating to criminal convictions or offences as defined in Article 10. An example would be a general hospital keeping patients' medical records or a private investigator keeping offenders' details. Beyond these provisions of the GDPR, some categories of data can be considered as increasing the possible risk to the rights and freedoms of individuals. These personal data are considered as sensitive (as this term is commonly understood) because they are linked to household and private activities (such as electronic communications whose confidentiality should be protected), or because they impact the exercise of a fundamental right (such as location data whose collection questions the freedom of movement) or because their violation clearly involves serious impacts in the data subject's daily life (such as financial data that might be used for payment fraud). In this regard, whether the data has already been made publicly available by the data subject or by third parties may be relevant. The fact that personal data is publicly available may be considered as a factor in the assessment if the data was expected to be further used for certain purposes. This criterion may also include data such as personal documents, emails, diaries, notes from e-readers equipped with note-taking features, and very personal information contained in life-logging applications.

5. Data processed on a large scale: the GDPR does not define what constitutes large-scale, though recital 91 provides some guidance. In any event, the WP29 recommends that the following factors, in particular, be considered when determining whether the processing is carried out on a large scale[^16]: <ol type = "a"><li> the number of data subjects concerned, either as a specific number or as a proportion of the relevant population; </li><li> the volume of data and/or the range of different data items being processed; </li><li> the duration, or permanence, of the data processing activity; </li><li> the geographical extent of the processing activity. </li></ol>

6. Matching or combining datasets, for example originating from two or more data processing operations performed for different purposes and/or by different data controllers in a way that would exceed the reasonable expectations of the data subject[^17].

7. Data concerning vulnerable data subjects (recital 75): the processing of this type of data is a criterion because of the increased power imbalance between the data subjects and the data controller, meaning the individuals may be unable to easily consent to, or oppose, the processing of their data, or exercise their rights. Vulnerable data subjects may include children (they can be considered as not able to knowingly and thoughtfully oppose or consent to the processing of their data), employees , more vulnerable segments of the population requiring special protection (mentally ill persons, asylum seekers, or the elderly, patients, etc.), and in any case where an imbalance in the relationship between the position of the data subject and the controller can be identified.

8. Innovative use or applying new technological or organisational solutions, like combining use of finger print and face recognition for improved physical access control, etc. The GDPR makes it clear (Article 35(1) and recitals 89 and 91) that the use of a new technology, defined in "accordance with the achieved state of technological knowledge" (recital 91), can trigger the need to carry out a DPIA. This is because the use of such technology can involve novel forms of data collection and usage, possibly with a high risk to individuals' rights and freedoms. Indeed, the personal and social consequences of the deployment of a new technology may be unknown. A DPIA will help the data controller to understand and to treat such risks. For example, certain "Internet of Things" applications could have a significant impact on individuals' daily lives and privacy; and therefore require a DPIA.

9. When the processing in itself "prevents data subjects from exercising a right or using a service or a contract" (Article 22 and recital 91). This includes processing operations that aims at allowing, modifying or refusing data subjects' access to a service or entry into a contract. An example of this is where a bank screens its customers against a credit reference database in order to decide whether to offer them a loan.

In most cases, a data controller can consider that a processing meeting two criteria would require a DPIA to be carried out. In general, the WP29 considers that the more criteria are met by the processing, the more likely it is to present a high risk to the rights and freedoms of data subjects, and therefore to require a DPIA, regardless of the measures which the controller envisages to adopt.

However, in some cases, a data controller can consider that a processing meeting only one of these criteria requires a DPIA.

The following examples illustrate how the criteria should be used to assess whether a particular processing operation requires a DPIA:

|Examples of processing | Possible Relevant criteria | DPIA likely to be required? |
|---|---|---|
|A hospital processing its patients' genetic and health data (hospital information system). | <ul><li> Sensitive data or data of a highly personal nature. </li><li> Data concerning vulnerable data subjects. </li><li> Data processed on a large-scale.</li></ol> | Yes |
| The use of a camera system to monitor driving behavior on highways. The controller envisages to use an intelligent video analysis system to single out cars and automatically recognize license plates. | <ul><li> Systematic monitoring. </li><li> Innovative use or applying technological or organisational solutions.</li></ol> |Yes|
|A company systematically monitoring its employees' activities, including the monitoring of the employees' work station, internet activity, etc. | <ul><li> Systematic monitoring. </li><li> Data concerning vulnerable data subjects. </li></ol> |Yes |
|The gathering of public social media data for generating profiles. |<ul><li> Evaluation or scoring. </li><li>Data processed on a large scale. </li><li> Matching or combining of datasets. </li><li> Sensitive data or data of a highly personal nature:</li></ol> | Yes |
|An institution creating a national level credit rating or fraud database. |<ul><li> Evaluation or scoring. </li><li> Automated decision making with legal or similar significant effect. </li><li> Prevents data subject from exercising a right or using a service or a contract. </li><li> Sensitive data or data of a highly personal nature: </li></ol> |Yes |
| Storage for archiving purpose of pseudonymised personal sensitive data concerning vulnerable data subjects of research projects or clinical trials |<ul><li> Sensitive data. </li><li> Data concerning vulnerable data subjects. - Prevents data subjects from exercising a right or using a service or a contract. </li></ol> |Yes |
| A processing of "personal data from patients or clients by an individual physician, other health care professional or lawyer" (Recital 91). |<ul><li> Sensitive data or data of a highly personal nature. - Data concerning vulnerable data subjects.</li></ol> | No |
|An online magazine using a mailing list to send a generic daily digest to its subscribers. | Data processed on a large scale. | No |
|An e-commerce website displaying adverts for vintage car parts involving limited profiling based on items viewed or purchased on its own website. | Evaluation or scoring. | No |

Conversely, a processing operation may correspond to the above mentioned cases and still be considered by the controller not to be "likely to result in a high risk". In such cases the controller should justify and document the reasons for not carrying out a DPIA, and include/record the views of the data protection officer.

In addition, as part of the accountability principle, every data controller "shall maintain a record of processing activities under its responsibility" including inter alia the purposes of processing, a description of the categories of data and recipients of the data and "where possible, a general description of the technical and organisational security measures referred to in Article 32(1)" (Article 30(1)) and must assess whether a high risk is likely, even if they ultimately decide not to carry out a DPIA.

Note: supervisory authorities are required to establish, make public and communicate a list of the processing operations that require a DPIA to the European Data Protection Board (EDPB) (Article 35(4))[^18]. The criteria set out above can help supervisory authorities to constitute such a list, with more specific content added in time if appropriate. For example, the processing of any type of biometric data or that of children could also be considered as relevant for the development of a list pursuant to article 35(4).

  
[^11]: See recitals 89, 91 and Article 35(1) and (3) for further examples.  
[^12]: See recital 71: "in particular analysing or predicting aspects concerning performance at work, economic situation, health, personal preferences or interests, reliability or behaviour, location or movements, in order to create or use personal profiles".  
[^13]: See recital 75: "where personal data are processed which reveal racial or ethnic origin, political opinions, religion or philosophical beliefs, trade union membership, and the processing of genetic data, data concerning health or data concerning sex life or criminal convictions and offences or related security measures".  
[^14]: See e.g. recitals 75, 76, 92, 116.  
[^15]: The WP29 interprets "systematic" as meaning one or more of the following (see the WP29 Guidelines on Data Protection Officer 16/EN WP 243): <ul><li> occurring according to a system; </li><li> pre-arranged, organised or methodical; </li><li> taking place as part of a general plan for data collection; </li><li> carried out as part of a strategy. </ul>The WP29 interprets "publicly accessible area" as being any place open to any member of the public, for example a piazza, a shopping centre, a street, a market place, a train station or a public library.  
[^16]: See the WP29 Guidelines on Data Protection Officer 16/EN WP 243.  
[^17]: See explanation in the WP29 Opinion on Purpose limitation 13/EN WP 203, p.24.  
[^18]: In that context, "the competent supervisory authority shall apply the consistency mechanism referred to in Article 63 where such lists involve processing activities which are related to the offering of goods or services to data subjects or to the monitoring of their behaviour in several Member States, or may substantially affect the free movement of personal data within the Union" (Article 35(6)).